In [4]:
import pandas as pd
import numpy as np
import logging
from math import floor
from google.oauth2.service_account import Credentials
import gspread
from typing import Dict, Any, List
from utils import get_teg_rounds, load_all_data, aggregate_data





In [ ]:
all_data = load_all_data(exclude_teg_50=True,exclude_incomplete_tegs=True)
print(all_data)

In [6]:
all_data = load_all_data(exclude_teg_50=True,exclude_incomplete_tegs=True)
agg_measures = ['Sc','GrossVP','NetVP','Stableford']
teg_data = aggregate_data(all_data,'TEG',agg_measures)

teg_data

Group columns: ['Pl', 'TEG', 'Player', 'TEGNum']
DataFrame columns: ['TEG', 'Round', 'Hole', 'PAR', 'SI', 'Pl', 'Sc', 'HC', 'HCStrokes', 'GrossVP', 'Net', 'NetVP', 'Stableford', 'TEGNum', 'HoleID', 'Player', 'FrontBack', 'Date', 'Course', 'Hole Order Ever', 'Sc Cum Round', 'Sc Cum TEG', 'Sc Cum Career', 'GrossVP Cum Round', 'GrossVP Cum TEG', 'GrossVP Cum Career', 'NetVP Cum Round', 'NetVP Cum TEG', 'NetVP Cum Career', 'Stableford Cum Round', 'Stableford Cum TEG', 'Stableford Cum Career', 'TEG Count', 'Career Count', 'Sc Round Avg', 'Sc TEG Avg', 'Sc Career Avg', 'GrossVP Round Avg', 'GrossVP TEG Avg', 'GrossVP Career Avg', 'NetVP Round Avg', 'NetVP TEG Avg', 'NetVP Career Avg', 'Stableford Round Avg', 'Stableford TEG Avg', 'Stableford Career Avg', 'Year']


,Pl,TEG,Player,TEGNum,Sc,GrossVP,NetVP,Stableford
0,AB,TEG 10,Alex BAKER,10,376.0,90.0,-38.0,184.0
1,AB,TEG 11,Alex BAKER,11,394.0,106.0,10.0,137.0
2,AB,TEG 12,Alex BAKER,12,392.0,104.0,4.0,143.0
3,AB,TEG 13,Alex BAKER,13,390.0,103.0,3.0,141.0
4,AB,TEG 14,Alex BAKER,14,389.0,102.0,-2.0,152.0
...,...,...,...,...,...,...,...,...
70,SN,TEG 5,Stuart NEUMANN,5,373.0,88.0,8.0,139.0
71,SN,TEG 6,Stuart NEUMANN,6,386.0,97.0,9.0,139.0
72,SN,TEG 7,Stuart NEUMANN,7,358.0,68.0,-24.0,169.0
73,SN,TEG 8,Stuart NEUMANN,8,422.0,137.0,65.0,100.0


In [8]:
df = teg_data

best_stableford = df.loc[df.groupby('Player')['Stableford'].idxmax()]

# Select only relevant columns for output
output = best_stableford[['Player', 'Stableford', 'TEG']].rename(columns={'Stableford': 'Best Stableford'})

# Display output
print(output)

            Player  Best Stableford     TEG
0       Alex BAKER            184.0  TEG 10
21    David MULLIN            170.0   TEG 7
24  Gregg WILLIAMS            173.0  TEG 10
42    Henry MELLER            175.0   TEG 7
60  John PATTERSON            172.0  TEG 12
51       Jon BAKER            164.0   TEG 3
72  Stuart NEUMANN            169.0   TEG 7


In [10]:

# Find unique combinations of TEG and Year
unique_teg_year = all_data[['TEG', 'Year']].drop_duplicates()

# Merge teg_data with the unique TEG-Year combinations to add the Year column
df_merged = pd.merge(teg_data, unique_teg_year, on='TEG', how='left')

# Define metrics and aggregation functions
metrics = {
    #'lowest Sc': 'Sc',
    'lowest GrossVP': 'GrossVP',
    'lowest NetVP': 'NetVP',
    'highest Stableford': 'Stableford'
}

# Initialize a dictionary to store outputs
output_dict = {}

# Loop over each metric and create the desired DataFrame
for metric_name, column in metrics.items():
    # Set up whether to find max or min for each metric
    if column == 'Stableford':
        best_values = df_merged.loc[df_merged.groupby('Player')[column].idxmax()]
    else:
        best_values = df_merged.loc[df_merged.groupby('Player')[column].idxmin()]

    # Select relevant columns and rename them
    output = best_values[['Player', column, 'TEG', 'Year']].rename(columns={column: metric_name})

    # Store the result in the dictionary
    output_dict[metric_name] = output
    
    # Print the output DataFrame
    print(f"Output for {metric_name}:")
    print(output, "\n")

Output for lowest GrossVP:
            Player  lowest GrossVP     TEG  Year
0       Alex BAKER            90.0  TEG 10  2017
10    David MULLIN            51.0  TEG 10  2017
29  Gregg WILLIAMS            63.0  TEG 15  2022
42    Henry MELLER            69.0   TEG 7  2014
63  John PATTERSON            88.0   TEG 9  2016
47       Jon BAKER            50.0  TEG 13  2020
72  Stuart NEUMANN            68.0   TEG 7  2014 

Output for lowest NetVP:
            Player  lowest NetVP     TEG  Year
0       Alex BAKER         -38.0  TEG 10  2017
21    David MULLIN         -23.0   TEG 7  2014
24  Gregg WILLIAMS         -28.0  TEG 10  2017
42    Henry MELLER         -31.0   TEG 7  2014
60  John PATTERSON         -20.0  TEG 12  2019
51       Jon BAKER         -18.0   TEG 3  2010
72  Stuart NEUMANN         -24.0   TEG 7  2014 

Output for highest Stableford:
            Player  highest Stableford     TEG  Year
0       Alex BAKER               184.0  TEG 10  2017
21    David MULLIN               170.0 

In [11]:
aggregate_data(data=all_data,aggregation_level='TEG')

Group columns: ['Pl', 'TEG', 'Player', 'TEGNum']
DataFrame columns: ['TEG', 'Round', 'Hole', 'PAR', 'SI', 'Pl', 'Sc', 'HC', 'HCStrokes', 'GrossVP', 'Net', 'NetVP', 'Stableford', 'TEGNum', 'HoleID', 'Player', 'FrontBack', 'Date', 'Course', 'Hole Order Ever', 'Sc Cum Round', 'Sc Cum TEG', 'Sc Cum Career', 'GrossVP Cum Round', 'GrossVP Cum TEG', 'GrossVP Cum Career', 'NetVP Cum Round', 'NetVP Cum TEG', 'NetVP Cum Career', 'Stableford Cum Round', 'Stableford Cum TEG', 'Stableford Cum Career', 'TEG Count', 'Career Count', 'Sc Round Avg', 'Sc TEG Avg', 'Sc Career Avg', 'GrossVP Round Avg', 'GrossVP TEG Avg', 'GrossVP Career Avg', 'NetVP Round Avg', 'NetVP TEG Avg', 'NetVP Career Avg', 'Stableford Round Avg', 'Stableford TEG Avg', 'Stableford Career Avg', 'Year']


,Pl,TEG,Player,TEGNum,Sc,GrossVP,NetVP,Stableford
0,AB,TEG 10,Alex BAKER,10,376.0,90.0,-38.0,184.0
1,AB,TEG 11,Alex BAKER,11,394.0,106.0,10.0,137.0
2,AB,TEG 12,Alex BAKER,12,392.0,104.0,4.0,143.0
3,AB,TEG 13,Alex BAKER,13,390.0,103.0,3.0,141.0
4,AB,TEG 14,Alex BAKER,14,389.0,102.0,-2.0,152.0
...,...,...,...,...,...,...,...,...
70,SN,TEG 5,Stuart NEUMANN,5,373.0,88.0,8.0,139.0
71,SN,TEG 6,Stuart NEUMANN,6,386.0,97.0,9.0,139.0
72,SN,TEG 7,Stuart NEUMANN,7,358.0,68.0,-24.0,169.0
73,SN,TEG 8,Stuart NEUMANN,8,422.0,137.0,65.0,100.0


In [ ]:
import pandas as pd
from IPython.display import Markdown

# Example DataFrame
data = {
    'Player': ['Alice', 'Bob', 'Charlie'],
    'Score': [72, 68, 74],
    'TEG': ['TEG1', 'TEG2', 'TEG1'],
    'Year': [2021, 2020, 2022],
    'Measure': ['Gross', 'Net', 'Stableford']
}

df = pd.DataFrame(data)

# Loop through DataFrame and display markdown text with dynamic content
for index, row in df.iterrows():
    # Define the markdown text using row values
    markdown_text = f"Best {row['Measure']}: **{row['Score']}** | {row['Player']} ({row['TEG']}, {row['Year']})"
    
    # Display the markdown text in Jupyter notebook
    display(Markdown(markdown_text))